In [271]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [272]:
df = pd.read_csv('data.csv')
df.head()
# df.shape

C:\Users\HP\AppData\Local\Temp\ipykernel_12984\660926661.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


,Article_id,User_id,inlink,externallink,comment,databyte
0,6,4477979,0,0,1,5
1,8,4477979,0,0,1,6
2,10,99,0,0,0,8
3,10,4477979,0,0,1,6
4,12,43,59,12,0,1468


In [273]:
# Selected only first 2 columns
df1 = df.iloc[:, 0:2]

# select first and 3rd columns
df2 = df.iloc[:, [0, 2]]

# df = df.drop(df.columns[0], axis=1)
df1.head()

,Article_id,User_id
0,6,4477979
1,8,4477979
2,10,99
3,10,4477979
4,12,43


In [274]:
# club rows which have same user_id and count the number of unique artical_id

df1 = df1.groupby(['Article_id']).nunique()

# df1 = df1.sort_values(by=['User_id'], ascending=False)

df1.head()

,User_id
Article_id,
6,1
8,1
10,2
12,936
23,1


In [275]:
# select 0 and 2 columns
df2 = df.iloc[:, [0, 2]]
df2.head()


,Article_id,inlink
0,6,0
1,8,0
2,10,0
3,10,0
4,12,59


In [276]:
df2 = df2.groupby(['Article_id'], group_keys=False).max()

df2.reset_index(inplace=True)

## if value of Article_id is more than 3000 then set it to random value till 3000

df2.loc[df2['Article_id'] > 3000, 'Article_id'] = np.random.randint(1, 3000, size=len(df2[df2['Article_id'] > 3000]))

df2['Article_id'].max()
# df2.columns

# merge first two title columns 

3000

In [277]:
df3 = df.iloc[:, [0, 3]]
df3.head()

,Article_id,externallink
0,6,0
1,8,0
2,10,0
3,10,0
4,12,12


In [278]:
df3 = df3.groupby('Article_id').max()
df3.head()

,externallink
Article_id,
6,0
8,0
10,0
12,118
23,0


In [279]:
df4 = df.iloc[:, [0, 4]]
df4 = df4.groupby('Article_id').max()
df4.head()

,comment
Article_id,
6,1
8,1
10,1
12,1
23,0


In [280]:
df7 = df4 = df.iloc[:, [0, 5]]

df8 = df7
df9 = df7

df7 = df7.groupby('Article_id').sum()
df8 = df8.groupby('Article_id').max()
df9 = df9.groupby('Article_id').count()


df7 = df7.join(df8, lsuffix='_sum', rsuffix='_max')
df7 = df7.join(df9, lsuffix='_sum', rsuffix='_count')


df7.head()

,databyte _sum,databyte _max,databyte
Article_id,,,
6,5,5,1
8,6,6,1
10,14,8,2
12,96824029,20211,10778
23,324,324,1


In [281]:
# merge all the dataframes
df6 = pd.merge(df1, df2, on='Article_id')
# df6 = pd.merge(df6, df3, on='Article_id')
df6 = pd.merge(df6, df7, on='Article_id')
# df6 = pd.merge(df6, df9, on='Article_id')

# rename the column User_id to User_count
df6 = df6.rename(columns={'User_id': 'user_count'})
df6 = df6.rename(columns={'databyte  _sum': 'databyte_sum'})
df6 = df6.rename(columns={'databyte  _max': 'databyte_max'})
df6 = df6.rename(columns={'databyte  ': 'databyte'})

# df6 = pd.merge(df6, df4, on='Article_id')
# df6 = pd.merge(df6, df5, on='Article_id')

df6.head()

,Article_id,user_count,inlink,databyte_sum,databyte_max,databyte
0,6,1,0,5,5,1
1,6,1,25,5,5,1
2,8,1,0,6,6,1
3,10,2,0,14,8,2
4,12,936,1125,96824029,20211,10778


In [282]:

# get column names
# df6.columns


df6['credibility'] = ((df6['user_count'])**2) + (df6['inlink']**2) + (((df6['databyte_sum']/df6['databyte'])*df6['databyte_max'])**2) 




# max value of credibility
# df6['credibility'].max()

df6.head()

,Article_id,user_count,inlink,databyte_sum,databyte_max,databyte,credibility
0,6,1,0,5,5,1,6.260000e+02
1,6,1,25,5,5,1,1.251000e+03
2,8,1,0,6,6,1,1.297000e+03
3,10,2,0,14,8,2,3.140000e+03
4,12,936,1125,96824029,20211,10778,3.296595e+16


In [295]:
df7 = df6.drop(['user_count','inlink', 'databyte_sum','databyte_max','databyte'], axis=1)

# convert credibility column to big int
# df7['credibility'] = df7['credibility']

# remove index column
df7 = df7.reset_index(drop=True)

# sqrt of credibility
df7['credibility'] = np.sqrt(df7['credibility'])
df7['credibility'] = df7['credibility'].astype('int64')

df7.head()

,Article_id,credibility
0,6,25
1,6,35
2,8,36
3,10,56
4,12,181565267


In [293]:
# Export the dataframe to csv file without index

df7.to_csv('credibility.csv', index=False)
